In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/llm-classification-finetuning/sample_submission.csv
/kaggle/input/llm-classification-finetuning/train.csv
/kaggle/input/llm-classification-finetuning/test.csv


In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch


2025-07-31 12:54:18.957110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753966459.216622      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753966459.291703      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
test_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")
sample_sub = pd.read_csv("/kaggle/input/llm-classification-finetuning/sample_submission.csv")

In [4]:
train_df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [5]:
def get_label(row):
    if row["winner_model_a"] == 1:
        return 0
    elif row["winner_model_b"] == 1:
        return 1
    else:
        return 2

train_df["label"] = train_df.apply(get_label, axis=1)


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=30000, stop_words="english")),
    ("clf", LogisticRegression(max_iter=1000, multi_class="multinomial"))
])

train_df["text"] = train_df["prompt"] + " [A] " + train_df["response_a"] + " [B] " + train_df["response_b"]
train_df["label"] = train_df[["winner_model_a", "winner_model_b", "winner_tie"]].values.argmax(axis=1)

pipeline.fit(train_df["text"], train_df["label"])

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=30000, stop_words='english')),
                ('clf',
                 LogisticRegression(max_iter=1000, multi_class='multinomial'))])

In [7]:
test_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")
test_df["text"] = test_df["prompt"] + " [A] " + test_df["response_a"] + " [B] " + test_df["response_b"]

In [8]:
pred_probs = pipeline.predict_proba(test_df["text"])

In [9]:
submission = pd.DataFrame({
    "id": test_df["id"],
    "winner_model_a": pred_probs[:, 0],
    "winner_model_b": pred_probs[:, 1],
    "winner_tie": pred_probs[:, 2]
})
submission.to_csv("submission.csv", index=False)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

X_train, X_val, y_train, y_val = train_test_split(train_df["text"], train_df["label"], test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)
val_probs = pipeline.predict_proba(X_val)
print("Validation Log Loss:", log_loss(y_val, val_probs))


Validation Log Loss: 1.120184188658189
